# Lesson 3: Exercise 1 Solution - Extract and Transform Trips from PostgreSQL

## Goal

Extract trips data from PostgreSQL and transform it into a **staging format** ready for warehouse loading. This is the first step in our ETL pipeline.

## Prerequisites

You should have completed:
- **Lesson 1, Exercise 1**: Connected to PostgreSQL and previewed the `raw_trips` table
- **Lesson 2, Exercise 1**: Designed the `dw_dim_rider` table in Redshift

## What You Will Build

A Pandas-based ETL script that:

1. Connects to PostgreSQL and extracts trips data
2. Cleans and standardizes fields (whitespace, nulls, data types)
3. Validates the transformation
4. Outputs to staging format (CSV/Parquet)


### Acceptance Criteria

- Trips data is extracted from PostgreSQL `raw_trips` table
- Data is cleaned and transformed
- Output matches the `stg_trips_raw` schema structure
- Row counts are preserved (no data loss)
- Output file is created successfully

---

## Lesson 3 Exercise 1: Extract and Transform Trips from PostgreSQL Solution

## Imports and Dependencies

Run this cell first to import all required libraries.

In [1]:
# ========= Imports
import os
from datetime import datetime
from typing import Tuple, List
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, text

print("All imports successful!")
print(f"   - pandas version: {pd.__version__}")
print(f"   - numpy version: {np.__version__}")

All imports successful!
   - pandas version: 2.3.1
   - numpy version: 2.2.6


---
## Configuration

**Important:** These credentials match the `populate-postgres.py` script from Lesson 1. Update only if your environment differs.

In [2]:
# ========= PostgreSQL Configuration ==========
# These match the populate-postgres.py script from Lesson 1.
# Only change if your environment is different.

PG_HOST = "localhost"      # Database host
PG_PORT = "5432"           # Database port
PG_DB = "postgres"         # Database name (populate script uses 'postgres')
PG_USER = "temp"           # User from populate-postgres.py
PG_PASSWORD = "temp"       # Password from populate-postgres.py

# Build connection URI
PG_URI = f"postgresql://{PG_USER}:{PG_PASSWORD}@{PG_HOST}:{PG_PORT}/{PG_DB}"

# Output paths
OUTPUT_STAGING_CSV = "/tmp/stg_trips_raw.csv"
OUTPUT_STAGING_PARQUET = "/tmp/stg_trips_raw.parquet"

print("Configuration set!")
print(f"   - PostgreSQL: {PG_HOST}:{PG_PORT}/{PG_DB}")
print(f"   - User: {PG_USER}")
print(f"   - Output CSV: {OUTPUT_STAGING_CSV}")
print(f"   - Output Parquet: {OUTPUT_STAGING_PARQUET}")

Configuration set!
   - PostgreSQL: localhost:5432/postgres
   - User: temp
   - Output CSV: /tmp/stg_trips_raw.csv
   - Output Parquet: /tmp/stg_trips_raw.parquet


---
## Verify Database Setup

This cell verifies that the `raw_trips` table exists and has data. If verification fails, you need to run the `populate-postgres.py` script from Lesson 1:

```python
!python populate-postgres.py
```

In [3]:
# ========= Verify Database Setup ==========
# This checks that raw_trips exists and has data from Lesson 1.

def verify_postgres_setup():
    """Verify PostgreSQL has the raw_trips table with data."""
    print("Verifying PostgreSQL setup...")
    print("-" * 50)
    
    try:
        engine = create_engine(PG_URI)
        with engine.connect() as conn:
            # Test connection
            conn.execute(text("SELECT 1"))
            print(f"Connected to PostgreSQL ({PG_HOST}:{PG_PORT}/{PG_DB})")
            
            # Check if raw_trips exists
            result = conn.execute(text(
                "SELECT EXISTS (SELECT 1 FROM information_schema.tables WHERE table_name = 'raw_trips')"
            ))
            table_exists = result.scalar()
            
            if not table_exists:
                print("\n*** ERROR: Table 'raw_trips' does not exist! ***")
                print("\nPlease run the populate script from Lesson 1:")
                print("   python populate-postgres.py")
                engine.dispose()
                return False
            
            # Check row count
            result = conn.execute(text("SELECT COUNT(*) FROM raw_trips"))
            count = result.scalar()
            
            if count == 0:
                print("\n*** ERROR: Table 'raw_trips' is empty! ***")
                print("\nPlease run the populate script from Lesson 1:")
                print("   python populate-postgres.py")
                engine.dispose()
                return False
            
            print(f"Table 'raw_trips' exists with {count:,} rows")
            
            # Show sample
            result = conn.execute(text("SELECT trip_id, rider_id, total_fare_cad FROM raw_trips LIMIT 3"))
            print("\nSample data:")
            for row in result:
                print(f"   {row}")
        
        engine.dispose()
        print("\nVerification PASSED - ready to proceed!")
        return True
        
    except Exception as e:
        print(f"\n*** ERROR: Could not connect to PostgreSQL! ***")
        print(f"Details: {e}")
        print("\nTroubleshooting:")
        print("  1. Is PostgreSQL running?")
        print("  2. Check credentials in the Configuration cell above")
        print("  3. Run: python populate-postgres.py")
        return False

# Run verification
verify_postgres_setup()

Verifying PostgreSQL setup...
--------------------------------------------------
Connected to PostgreSQL (localhost:5432/postgres)
Table 'raw_trips' exists with 2,500 rows

Sample data:
   ('T100000', 'R33247', Decimal('3.32'))
   ('T100001', 'R43159', Decimal('3.17'))
   ('T100002', 'R18110', Decimal('2.12'))

Verification PASSED - ready to proceed!


True

---
## Column Specification

Define the expected columns and their types for the staging table. This matches the structure we'll use in Redshift.

In [4]:
# ========= Column specs for staging (name, kind)
# kind: 's' = string, 'ts' = timestamp, 'i' = integer, 'f' = float, 'b' = boolean

TRIPS_COLSPEC = [
    ('trip_id', 's'),
    ('rider_id', 's'),
    ('route_id', 's'),
    ('mode', 's'),
    ('origin_station_id', 's'),
    ('destination_station_id', 's'),
    ('board_datetime', 'ts'),
    ('alight_datetime', 'ts'),
    ('country', 's'),
    ('province', 's'),
    ('fare_class', 's'),
    ('payment_method', 's'),
    ('transfers', 'i'),
    ('zones_charged', 'i'),
    ('distance_km', 'f'),
    ('base_fare_cad', 'f'),
    ('discount_rate', 'f'),
    ('discount_amount_cad', 'f'),
    ('yvr_addfare_cad', 'f'),
    ('total_fare_cad', 'f'),
    ('on_time_arrival', 'b'),
    ('service_disruption', 'b'),
    ('polyline_stations', 's'),
]

print(f"Column spec defined: {len(TRIPS_COLSPEC)} columns")
print("\nColumn breakdown:")
print(f"   - String columns: {sum(1 for _, k in TRIPS_COLSPEC if k == 's')}")
print(f"   - Timestamp columns: {sum(1 for _, k in TRIPS_COLSPEC if k == 'ts')}")
print(f"   - Integer columns: {sum(1 for _, k in TRIPS_COLSPEC if k == 'i')}")
print(f"   - Float columns: {sum(1 for _, k in TRIPS_COLSPEC if k == 'f')}")
print(f"   - Boolean columns: {sum(1 for _, k in TRIPS_COLSPEC if k == 'b')}")

Column spec defined: 23 columns

Column breakdown:
   - String columns: 11
   - Timestamp columns: 2
   - Integer columns: 2
   - Float columns: 6
   - Boolean columns: 2


---
## Helper Functions

These functions are adapted from the project solution for cleaning and transforming data.

In [5]:
# ========= Helper Functions (from project solution)

def trim_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Standardize text fields (strip whitespace) and handle NaN values properly.
    
    This is the same helper used in the final project for cleaning extracted data.
    
    Args:
        df: Input DataFrame with potentially messy string fields
    
    Returns:
        Cleaned DataFrame with stripped strings and proper NaN handling
    """
    df = df.copy()  # Avoid modifying the original DataFrame
    for c in df.select_dtypes(include=['object']).columns:
        # Convert to string and strip whitespace
        df[c] = df[c].astype(str).str.strip()
        # Replace 'nan', 'None', and empty strings with actual NaN
        df[c] = df[c].mask(df[c].isin(['nan', 'None', 'NaN', '']), np.nan)
    return df


def validate_columns(df: pd.DataFrame, colspec: List[Tuple[str, str]]) -> dict:
    """
    Validate that DataFrame columns match the expected spec.
    
    Args:
        df: DataFrame to validate
        colspec: List of (column_name, type_code) tuples
    
    Returns:
        Dict with validation results
    """
    expected_cols = {c for c, _ in colspec}
    actual_cols = set(df.columns)
    
    return {
        'missing': expected_cols - actual_cols,
        'extra': actual_cols - expected_cols,
        'matched': expected_cols & actual_cols,
        'valid': expected_cols == actual_cols
    }


print("Helper functions defined: trim_df(), validate_columns()")

Helper functions defined: trim_df(), validate_columns()


---
## Step 1: Connect to PostgreSQL

Establish a connection to the PostgreSQL database using SQLAlchemy (same pattern as Lesson 1, Exercise 1).

In [6]:
# ========= STEP 1: Connect to PostgreSQL
print("Step 1: Connecting to PostgreSQL...")
print("-" * 50)

engine = create_engine(PG_URI)

# Test the connection
with engine.connect() as conn:
    result = conn.execute(text("SELECT 1"))
    print("Successfully connected to PostgreSQL!")
    
    # Get row count
    count_result = conn.execute(text("SELECT COUNT(*) FROM raw_trips"))
    total_rows = count_result.scalar()
    print(f"Total rows in raw_trips: {total_rows:,}")

Step 1: Connecting to PostgreSQL...
--------------------------------------------------
Successfully connected to PostgreSQL!
Total rows in raw_trips: 2,500


---
## Step 2: Extract Data from PostgreSQL

Pull all trips data from the `raw_trips` table. In production ETL, you might add filters for incremental loads.

In [7]:
# ========= STEP 2: Extract from PostgreSQL
print("Step 2: Extracting trips data from PostgreSQL...")
print("-" * 50)

# SQL to extract all trips
# In production, you might add WHERE clauses for incremental loads
SQL_EXTRACT = """
SELECT 
    trip_id,
    rider_id,
    route_id,
    mode,
    origin_station_id,
    destination_station_id,
    board_datetime,
    alight_datetime,
    country,
    province,
    fare_class,
    payment_method,
    transfers,
    zones_charged,
    distance_km,
    base_fare_cad,
    discount_rate,
    discount_amount_cad,
    yvr_addfare_cad,
    total_fare_cad,
    on_time_arrival,
    service_disruption,
    polyline_stations
FROM raw_trips
"""

with engine.connect() as conn:
    trips_raw = pd.read_sql(text(SQL_EXTRACT), conn)

print(f"Extracted {len(trips_raw):,} rows")
print(f"Columns: {len(trips_raw.columns)}")
print(f"\nSample data (first 3 rows):")
display(trips_raw.head(3))

Step 2: Extracting trips data from PostgreSQL...
--------------------------------------------------
Extracted 2,500 rows
Columns: 23

Sample data (first 3 rows):


,trip_id,rider_id,route_id,mode,origin_station_id,destination_station_id,board_datetime,alight_datetime,country,province,...,zones_charged,distance_km,base_fare_cad,discount_rate,discount_amount_cad,yvr_addfare_cad,total_fare_cad,on_time_arrival,service_disruption,polyline_stations
0,T100000,R33247,R111,bus,S021,S004,2024-01-31 10:45:08,2024-01-31 11:12:09,CA,BC,...,1,7.26,3.32,0.00,0.0,0.0,3.32,True,False,S001|S025|S009|S008|S009
1,T100001,R43159,R033,bus,S005,S025,2024-08-08 00:16:41,2024-08-08 00:44:35,CA,BC,...,1,11.66,3.17,0.00,0.0,0.0,3.17,True,False,S004|S023|S025|S030|S018|S003|S020
2,T100002,R18110,R001,bus,S014,S002,2024-05-28 02:42:12,2024-05-28 03:14:48,CA,BC,...,1,15.35,3.12,0.32,1.0,0.0,2.12,False,False,S001|S004|S008|S009|S018|S021|S001|S019|S007


---
## Step 3: Transform and Clean Data

Apply transformations to prepare the data for staging:
- Strip whitespace from string fields
- Standardize NULL representations
- Parse timestamps
- Ensure proper data types

In [8]:
# ========= STEP 3: Transform and clean
print("Step 3: Transforming and cleaning data...")
print("-" * 50)

# Apply string cleaning
trips_clean = trim_df(trips_raw)
print("Applied trim_df() - whitespace stripped, NaN standardized")

# Parse timestamp columns (if not already datetime)
timestamp_cols = [c for c, k in TRIPS_COLSPEC if k == 'ts']
for col in timestamp_cols:
    if col in trips_clean.columns:
        if not pd.api.types.is_datetime64_any_dtype(trips_clean[col]):
            trips_clean[col] = pd.to_datetime(trips_clean[col], errors='coerce')
            print(f"   Parsed timestamp: {col}")

# Ensure boolean columns are proper booleans
boolean_cols = [c for c, k in TRIPS_COLSPEC if k == 'b']
for col in boolean_cols:
    if col in trips_clean.columns:
        # Handle various boolean representations
        if trips_clean[col].dtype == 'object':
            trips_clean[col] = trips_clean[col].map(
                lambda x: True if str(x).lower() in ('true', 't', '1', 'yes') 
                          else (False if str(x).lower() in ('false', 'f', '0', 'no') else None)
            )
            print(f"   Converted boolean: {col}")

# Ensure numeric columns are proper types
float_cols = [c for c, k in TRIPS_COLSPEC if k == 'f']
for col in float_cols:
    if col in trips_clean.columns:
        trips_clean[col] = pd.to_numeric(trips_clean[col], errors='coerce')

int_cols = [c for c, k in TRIPS_COLSPEC if k == 'i']
for col in int_cols:
    if col in trips_clean.columns:
        trips_clean[col] = pd.to_numeric(trips_clean[col], errors='coerce').astype('Int64')

print(f"\nTransformation complete!")
print(f"   - Rows: {len(trips_clean):,}")
print(f"   - Columns: {len(trips_clean.columns)}")

Step 3: Transforming and cleaning data...
--------------------------------------------------
Applied trim_df() - whitespace stripped, NaN standardized

Transformation complete!
   - Rows: 2,500
   - Columns: 23


In [ ]:
# Inspect data types after transformation
print("Data types after transformation:")
print("-" * 50)
print(trips_clean.dtypes)

---
## Step 4: Validate the Transformation

Before outputting, verify that:
- All expected columns are present
- Row counts match (no data loss)
- Key fields have no unexpected nulls

In [9]:
# ========= STEP 4: Validate
print("Step 4: Validating transformation...")
print("-" * 50)

# Check column alignment
validation = validate_columns(trips_clean, TRIPS_COLSPEC)

if validation['valid']:
    print("Column validation: PASSED")
else:
    print("Column validation: ISSUES FOUND")
    if validation['missing']:
        print(f"   Missing columns: {validation['missing']}")
    if validation['extra']:
        print(f"   Extra columns: {validation['extra']}")

# Check row counts
print(f"\nRow count check:")
print(f"   Source rows: {len(trips_raw):,}")
print(f"   Output rows: {len(trips_clean):,}")
print(f"   Match: {'YES' if len(trips_raw) == len(trips_clean) else 'NO - DATA LOSS!'}")

# Check for nulls in key fields
key_fields = ['trip_id', 'rider_id', 'route_id']
print(f"\nNull check for key fields:")
for field in key_fields:
    null_count = trips_clean[field].isna().sum()
    print(f"   {field}: {null_count} nulls ({null_count/len(trips_clean)*100:.2f}%)")

Step 4: Validating transformation...
--------------------------------------------------
Column validation: PASSED

Row count check:
   Source rows: 2,500
   Output rows: 2,500
   Match: YES

Null check for key fields:
   trip_id: 0 nulls (0.00%)
   rider_id: 0 nulls (0.00%)
   route_id: 0 nulls (0.00%)


In [10]:
# Summary statistics for numeric fields
print("Summary statistics for key measures:")
print("-" * 50)
display(trips_clean[['distance_km', 'total_fare_cad', 'transfers', 'zones_charged']].describe())

Summary statistics for key measures:
--------------------------------------------------


,distance_km,total_fare_cad,transfers,zones_charged
count,2500.000000,2500.000000,2500.0,2500.0
mean,10.854608,3.028128,0.3868,1.1376
std,5.058530,0.941073,0.5971,0.344549
min,0.800000,1.860000,0.0,1.0
25%,7.637500,2.170000,0.0,1.0
50%,10.110000,3.130000,0.0,1.0
75%,12.922500,3.190000,1.0,1.0
max,31.810000,9.580000,2.0,2.0


---
## Step 5: Output to Staging Format

Save the transformed data in formats suitable for warehouse loading:
- **CSV**: Human-readable, compatible with Redshift COPY
- **Parquet**: Compressed, columnar format for efficient loading

In [12]:
# ========= STEP 5: Output to staging format
print("Step 5: Outputting to staging format...")
print("-" * 50)

# Select only the columns defined in the spec (in order)
output_cols = [c for c, _ in TRIPS_COLSPEC]
trips_staging = trips_clean[output_cols]

# Output to CSV
trips_staging.to_csv(OUTPUT_STAGING_CSV, index=False)
csv_size = os.path.getsize(OUTPUT_STAGING_CSV) / 1024  # KB
print(f"CSV saved: {OUTPUT_STAGING_CSV}")
print(f"   Size: {csv_size:.1f} KB")

# Output to Parquet (optional - requires pyarrow or fastparquet)
try:
    trips_staging.to_parquet(OUTPUT_STAGING_PARQUET, index=False)
    parquet_size = os.path.getsize(OUTPUT_STAGING_PARQUET) / 1024  # KB
    print(f"\nParquet saved: {OUTPUT_STAGING_PARQUET}")
    print(f"   Size: {parquet_size:.1f} KB")
    print(f"   Compression ratio: {csv_size/parquet_size:.1f}x")
except ImportError:
    print("\nParquet output skipped (pyarrow not installed).")
    print("   To enable: pip install pyarrow")
    print("   CSV output is sufficient for this exercise.")

Step 5: Outputting to staging format...
--------------------------------------------------
CSV saved: /tmp/stg_trips_raw.csv
   Size: 449.1 KB

Parquet output skipped (pyarrow not installed).
   To enable: pip install pyarrow
   CSV output is sufficient for this exercise.


---
## Step 6: Verify Output

Read back the output file to confirm it was written correctly.

In [13]:
# ========= STEP 6: Verify output
print("Step 6: Verifying output...")
print("-" * 50)

# Read back the CSV
trips_verify = pd.read_csv(OUTPUT_STAGING_CSV)

print(f"Read back {len(trips_verify):,} rows from CSV")
print(f"Columns match: {list(trips_verify.columns) == output_cols}")

# Show first few rows
print(f"\nFirst 3 rows of staged data:")
display(trips_verify.head(3))

Step 6: Verifying output...
--------------------------------------------------
Read back 2,500 rows from CSV
Columns match: True

First 3 rows of staged data:


,trip_id,rider_id,route_id,mode,origin_station_id,destination_station_id,board_datetime,alight_datetime,country,province,...,zones_charged,distance_km,base_fare_cad,discount_rate,discount_amount_cad,yvr_addfare_cad,total_fare_cad,on_time_arrival,service_disruption,polyline_stations
0,T100000,R33247,R111,bus,S021,S004,2024-01-31 10:45:08,2024-01-31 11:12:09,CA,BC,...,1,7.26,3.32,0.00,0.0,0.0,3.32,True,False,S001|S025|S009|S008|S009
1,T100001,R43159,R033,bus,S005,S025,2024-08-08 00:16:41,2024-08-08 00:44:35,CA,BC,...,1,11.66,3.17,0.00,0.0,0.0,3.17,True,False,S004|S023|S025|S030|S018|S003|S020
2,T100002,R18110,R001,bus,S014,S002,2024-05-28 02:42:12,2024-05-28 03:14:48,CA,BC,...,1,15.35,3.12,0.32,1.0,0.0,2.12,False,False,S001|S004|S008|S009|S018|S021|S001|S019|S007


---
## Step 7: Clean Up

Close the database connection.

In [14]:
# ========= STEP 7: Clean up
engine.dispose()
print("PostgreSQL connection closed.")

PostgreSQL connection closed.


---
## Summary

Generate a final summary report of the ETL job.

In [15]:
# ========= Final Summary
print("=" * 60)
print("ETL JOB SUMMARY: Trips to Staging")
print("=" * 60)
print(f"""
Source:           PostgreSQL {PG_HOST}:{PG_PORT}/{PG_DB}
Table:            raw_trips
Output CSV:       {OUTPUT_STAGING_CSV}
Output Parquet:   {OUTPUT_STAGING_PARQUET}

Records:
  - Extracted:    {len(trips_raw):,}
  - Transformed:  {len(trips_clean):,}
  - Staged:       {len(trips_staging):,}

Data Quality:
  - Columns:      {len(trips_staging.columns)} (all expected)
  - Null trip_id: {trips_staging['trip_id'].isna().sum()}
  - Date range:   {trips_staging['board_datetime'].min()} to {trips_staging['board_datetime'].max()}

Status:           SUCCESS
Completed:        {datetime.now().isoformat()}
""")
print("=" * 60)

ETL JOB SUMMARY: Trips to Staging

Source:           PostgreSQL localhost:5432/postgres
Table:            raw_trips
Output CSV:       /tmp/stg_trips_raw.csv
Output Parquet:   /tmp/stg_trips_raw.parquet

Records:
  - Extracted:    2,500
  - Transformed:  2,500
  - Staged:       2,500

Data Quality:
  - Columns:      23 (all expected)
  - Null trip_id: 0
  - Date range:   2024-01-01 01:08:58 to 2025-06-29 20:37:07

Status:           SUCCESS
Completed:        2025-12-15T19:15:38.852136

